# 파일 불러오기

In [1]:
import pandas as pd

customers = pd.read_csv('./data/olist_customers_dataset.csv')
geolocation = pd.read_csv('./data/olist_geolocation_dataset.csv')
orders = pd.read_csv('./data/olist_orders_dataset.csv')
order_items = pd.read_csv('./data/olist_order_items_dataset.csv')
order_payments = pd.read_csv('./data/olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('./data/reviews_translated.csv')
products = pd.read_csv('./data/olist_products_dataset.csv')
sellers = pd.read_csv('./data/olist_sellers_dataset.csv')
product_category_name_translation = pd.read_csv('./data/product_category_name_translation.csv')

c:\Users\yujeo\miniconda3\envs\ds_study\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\yujeo\miniconda3\envs\ds_study\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\yujeo\miniconda3\envs\ds_study\lib\site-packages\numpy\.libs\libopenblas.fb5ae2tyxyh2ijrdkgdgq3xbklktf43h.gfortran-win_amd64.dll
c:\Users\yujeo\miniconda3\envs\ds_study\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## 1. Orders 데이터

* nan값 제거

In [2]:
orders = orders.dropna() #null값 제거
orders.reset_index(inplace=True, drop=True)

* 컬럼 형변환 및 컬럼 추가

In [3]:
time_columns = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
orders[time_columns] = orders[time_columns].apply(pd.to_datetime) #시간 관련 컬럼 datetime으로 형변환

orders['estimate_delivery_time_diff'] = orders['order_estimated_delivery_date'] - orders['order_delivered_customer_date'] #배송 예측 시간 차이
orders['delivery_time'] = orders['order_estimated_delivery_date'] - orders['order_approved_at'] #배송시간

* delivery_time 음수 값 제거

In [4]:
orders = orders[orders['delivery_time'].dt.days>=0]
orders.reset_index(inplace=True, drop=True)

## 2. geolocation

* 위도,경도 통일
* 같은 우편번호에 위도,경도 값이 여러 개 존재함
* 좌표를 지도 상에 찍어보면 큰 차이가 없으므로 마지막 위도, 경도 값으로 대체

In [6]:
geolocation[geolocation['geolocation_zip_code_prefix']==1046]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
15,1046,-23.546081,-46.644820,sao paulo,SP
19,1046,-23.545884,-46.643163,sao paulo,SP
20,1046,-23.545166,-46.643786,sao paulo,SP
...,...,...,...,...,...
1432,1046,-23.545166,-46.643786,sao paulo,SP
1437,1046,-23.547191,-46.642923,sao paulo,SP
1439,1046,-23.545158,-46.643880,sao paulo,SP
1450,1046,-23.545292,-46.643130,sao paulo,SP


In [23]:
geolocation = geolocation.drop_duplicates(['geolocation_zip_code_prefix'], keep='last')
geolocation.reset_index(inplace=True, drop=True)

## 3. products

* nan값 제거

In [24]:
products = products.dropna()
products.reset_index(inplace=True, drop=True)

* product name 포르투갈어 -> 영어로 변경

In [25]:
products = products.merge(product_category_name_translation, on='product_category_name')
del products['product_category_name']
products.rename(columns={'product_category_name_english':'product_category_name'}, inplace=True)
products = products[['product_id', 'product_category_name', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']]

## 4. order_items

* 상품 주문 수량 컬럼 생성

In [26]:
#order_item_id를 제외하고 나머지 컬럼이 동일하다면 동일상품을 주문한 것이므로 order_item_id를 상품 주문 수량으로 생각하여 연산 
order_items = order_items.groupby(['order_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price','freight_value']).count()
order_items.rename(columns= {'order_item_id': 'order_count'}, inplace=True) #order_item_id 컬럼명을 order_count로 변경
order_items.reset_index(inplace=True)

* 불필요한 컬럼 제거

In [27]:
del order_items['shipping_limit_date']

## 5. order_reviews

* 불필요한 컬럼 제거

In [28]:
del order_reviews['Unnamed: 0']
del order_reviews['review_id']
del order_reviews['review_creation_date']
del order_reviews['review_answer_timestamp']

## 6. order_payments

### 다른 데이터와 merge할때 row의 중복이 많이 발생 -> 분할 결제 횟수만큼 row가 존재하여 merge할 경우 중복이 발생함
### order_id를 기준으로 그룹화하여 payment_value의 합계로 merge함 

#### order_id payment_value 총 합 = price * order_count + freight_value 

In [29]:
order_items[order_items['order_id']=='e481f51cbdc54678b7cc49136f2d6af7']

,order_id,product_id,seller_id,price,freight_value,order_count
91642,e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,1


In [30]:
order_payments[order_payments['order_id']=='e481f51cbdc54678b7cc49136f2d6af7'].groupby('order_id')['payment_value'].sum()

order_id
e481f51cbdc54678b7cc49136f2d6af7    38.71
Name: payment_value, dtype: float64

In [31]:
order_payments[order_payments['order_id']=='e481f51cbdc54678b7cc49136f2d6af7']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
10770,e481f51cbdc54678b7cc49136f2d6af7,1,credit_card,1,18.12
44246,e481f51cbdc54678b7cc49136f2d6af7,3,voucher,1,2.00
91130,e481f51cbdc54678b7cc49136f2d6af7,2,voucher,1,18.59


#### sum함수를 이용하여 그룹화

In [32]:
type_str = []
for i in order_payments['payment_type']: #sum을 했을 때, payment_type을 알아보기 쉽게 첫번째 알파벳만 표시
    if i == 'credit_card':
        type_str.append('c')
    elif i == 'boleto':
        type_str.append('b')
    elif i == 'voucher':
        type_str.append('v')
    elif i == 'debit_card':
        type_str.append('d')
    elif i == 'not_defined':
        type_str.append('n')

order_payments['payment_type_str'] = type_str

op = order_payments.groupby('order_id').aggregate({'payment_sequential':'max', 'payment_type_str':'sum', 'payment_installments':'max', 'payment_value':'sum'}) #그룹화
op.reset_index(inplace=True)

ts = []
for strings in op['payment_type_str']: #중복되는 문자열 제거
    x = "".join(sorted(set(strings), key=strings.index))
    ts.append(x)

op['payment_type_str'] = ts

for i,row in op.iterrows(): #알파벳을 원래 단어로 변경
    if row['payment_type_str'] =='c':
        op.iloc[i,2] = 'credit_card'
    elif row['payment_type_str'] == 'b':
        op.iloc[i,2] = 'boleto'
    elif row['payment_type_str'] == 'vc':
        op.iloc[i,2] = 'voucher, credit_card'
    elif row['payment_type_str'] == 'd':
        op.iloc[i,2] = 'debit_card'
    elif row['payment_type_str'] == 'n':
        op.iloc[i,2] = 'not_defined'
    elif row['payment_type_str'] == 'cv':
        op.iloc[i,2] = 'credit_card, voucher'
    elif row['payment_type_str'] == 'cd':
        op.iloc[i,2] = 'credit_card, debit_card'
    elif row['payment_type_str'] == 'v':
        op.iloc[i,2] = 'voucher'

op.rename(columns={'payment_type_str':'payment_type'}, inplace=True)

# 전체 데이터 merge

In [60]:
df = orders.merge(order_items, on='order_id') #order, order_items
df = df.merge(products, on='product_id') #orders, order_items, products
df = df.merge(customers, on='customer_id') #orders, order_items, products, customer
df = df.merge(order_reviews, on='order_id') #orders, order_items, products, customer, order_reviews
df = df.merge(sellers, on='seller_id') #orders, order_items, products, customer, order_reviews, sellers

df = df.merge(geolocation, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix') #customer zip_code 기준으로 병합
del df['geolocation_zip_code_prefix'] #중복컬럼 제거
del df['geolocation_city']
del df['geolocation_state']
df.rename(columns={'geolocation_lat':'customer_lat', 'geolocation_lng':'customer_lng'},inplace=True) #컬럼명 변경

df = df.merge(geolocation, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix') #seller zip_code 기준으로 병합
del df['geolocation_zip_code_prefix'] #중복컬럼 제거
del df['geolocation_city']
del df['geolocation_state']
df.rename(columns={'geolocation_lat':'seller_lat', 'geolocation_lng':'seller_lng'},inplace=True) #컬럼명 변경

df = df.merge(op, on='order_id')

In [61]:
df = df.drop_duplicates(keep='last')
df.reset_index(inplace=True, drop=True)

# 파일 저장

In [242]:
df.to_csv('./data/df_merged.csv', index=False)